In [1]:
class Pair:
	def __init__(self, x, y):
		self.x = x
		self.y = y
	
	def __repr__(self):
		return 'Pair({0.x!r}, {0.y!r})'.format(self)

	def __str__(self):
		return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(9,10)

In [7]:
print('p is {0!r}'.format(p))


p is Pair(9, 10)


In [2]:
_formats = {
	'ymd': '{d.year}-{d.month}-{d.day}',
	'mdy':'{d.month}-{d.day}-{d.year}',
	'dmy':'{d.day}-{d.month}-{d.year}'
}

class Date:
	def __init__(self, year, month, day):
		self.year = year
		self.month = month
		self.day = day

	def __format__(self, code):
		if code =='':
			code = 'ymd'
		fmt = _formats[code]
		return fmt.format(d=self)

In [3]:
d = Date(2012, 12, 21)
format(d)

'2012-12-21'

In [5]:
format(d, 'dmy')

'21-12-2012'

In [10]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConection:
	def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
		self.address = address
		self.family = family
		self.type = type
		self.sock = None

	def __enter__(self):
		if self.sock is not None:
			raise RuntimeError('Already Connected')
		self.sock = socket(self.family, self.type)
		self.sock.connect(self.address)
		return self.sock

	def __exit__(self, exc_ty, exc_val, tb):
		self.sock.close()
		self.sock = None

In [11]:
from functools import partial

conn = LazyConection(('www.python.org', 80))

with conn as s:
	s.send(b'GET /index.html HTTP/1.0\r\n')
	s.send(b'HOST: www.python.org\r\n')
	s.send(b'\r\n')
	resp = b''.join(iter(partial(s.recv, 8192), b''))

In [19]:
class Test:
	def __init__(self, val):
		self.val = val

	def add(self, to_add):
		self.val = self.val+ to_add

	def __enter__(self):
		print('__enter__ triggered')
		return self

	def __exit__(self, exc_ty, ext_val, tb):
		print('__exit__ triggered, ', exc_ty, ext_val, tb)
		return None

test = Test(123)

with test as t:
	print(t.val)
	t.add(100)
	print(t.val)

__enter__ triggered
123
223
__exit__ triggered,  None None None


In [16]:
class B:
	def __init__(self):
		self.__private = 0

	def __private_method(self):
		self.__private = 1

class C(B):
	def __init__(self):
		super().__init__()
		self.__private = 2
	
	def __private_method(self):
		self.__private = 3

b = B()

b._B__private_method()
b._B__private

1

In [19]:
class Person:
	def __init__(self, first_name):
		self.first_name = first_name

	# Getter function
	@property
	def first_name(self):
		print('Getting first name')
		return self._first_name
	
	# Setter function
	@first_name.setter
	def first_name(self, value):
		if not isinstance(value, str):
			raise TypeError('Expected a string')
		self._first_name = value
	
	# Deleter function (optional)
	@first_name.deleter
	def first_name(self):
		raise AttributeError("Can't delete attribute")

In [20]:
a = Person('Guido')

a.first_name

Getting first name


'Guido'

In [21]:
class Person2:
	def __init__(self, first_name):
		self.first_name = first_name


In [22]:
a = Person2('Guido')
a.first_name

'Guido'

In [27]:
class A:
	def spam(self):
		print('A.spam')

class B(A):
	def spam(self):
		print('B.spam')
		super().spam()

b = B()

b.spam()

B.spam
A.spam


In [1]:
class Person:
	def __init__(self, name):
		self.name = name
	
	# Getter function
	@property
	def name(self):
		return self._name

	# Setter function
	@name.setter
	def name(self, value):
		if not isinstance(value, str):
			raise TypeError('Expected a string')
		self._name = value

	# Deleter function
	@name.deleter
	def name(self):
		raise AttributeError('Can not delete attribute')
	

In [2]:
class SubPerson(Person):
	@property
	def name(self):
		print('Getting name')
		return super().name

	@name.setter
	def name(self, value):
		print('Setting name to', value)
		super(SubPerson, SubPerson).name.__set__(self, value)

	@name.deleter
	def name(self):
		print('Deleting name')
		super(SubPerson, SubPerson).name.__delete__(self)
		

In [3]:
s = SubPerson('Guido')



Setting name to Guido


In [4]:
s.name

Getting name


'Guido'

In [8]:
class String:
	def __init__(self, name):
		self.name = name
	
	def __get__(self, instance, cls):
		if instance is None:
			return self
		return instance.__dict__[self.name]

	def __set__(self, instance, value):
		if not isinstance(value, str):
			raise TypeError('Expected a string')
		instance.__dict__[self.name] = value

class Person:
	name = String('name')
	def __init__(self,name):
		self.name = name


class SubPerson(Person):
	@property
	def name(self):
		print('Getting Name')
		return super().name

	@name.setter
	def name(self, value):
		print("Setting name to", value)
		super(SubPerson, SubPerson).name.__set__(self, value)

	@name.deleter
	def name(self):
		print('Deleting name')
		super(SubPerson, SubPerson).name.__delete__(self)




In [13]:
p = SubPerson('e')

p.name

p.name = 'tim'

del p.name

Setting name to e
Getting Name
Setting name to tim
Deleting name


AttributeError: 'String' object has no attribute '__delete__'

In [1]:
# Descriptor attribute for an integer type-checked attribute
class Integer:
	def __init__(self, name):
		self.name = name

	def __get__(self, instance, cls):
		if instance is None:
			return self
		else:
			return instance.__dict__[self.name]

	def __set__(self, instance, value):
		if not isinstance(value, int):
			raise TypeError('Expected an int')
		instance.__dict__[self.name] = value 

	def __delete__(self, instance):
		del instance.__dict__[self.name]

class Point:
	x = Integer('x')
	y = Integer('y')
	def __init__(self, x, y):
		self.x = x
		self.y = y
	
p = Point(2,3)

p.x 


2

In [3]:
p.x = 2.2

TypeError: Expected an int

In [5]:
class lazyproperty:
	def __init__(self, func):
		self.func = func

	def __get__(self, instance, cls):
		if instance is None:
			return self
		else:
			value = self.func(instance)
			setattr(instance, self.func.__name__, value)
			return value

	
import math

class Circle:
	def __init__(self, radius):
		self.radius = radius

	@lazyproperty
	def area(self):
		print('Computing area')
		return math.pi * self.radius **2

	@lazyproperty
	def perimeter(self):
		print('Computing perimeter')
		return 2 * math.pi * self.radius

	

In [6]:
c = Circle(4.0)

c.radius

c.area

c.area

c.perimeter

c.perimeter


Computing area
Computing perimeter


25.132741228718345

In [8]:
class Structure:
	# Class variable that specifies expected fields
	_fields = []
	def __init__(self, *args):
		if len(args) != len(self._fields):
			raise TypeError('Expected {} arguments'.format(len(self._fields)))

		# Set the arguments
		for name, value in zip(self._fields, args):
			setattr(self, name, value)
			

In [9]:
class Stock(Structure):
	_fields = ['name', 'shares', 'price']

s = Stock('acme', 50, 91.1)

s.name

'acme'

In [ ]:
from abc import ABCMeta, abstractclassmethod, abstractmethod

class IStream(metaclass=ABCMeta):
	@abstractmethod
	def read(self, maxbytes=-1):
		pass
	@abstractmethod
	def write(self, data):
		pass

class SocketStream(IStream):
	def read(self, maxbyte=-1):
		...

	def write(self, data):
		...

		

In [2]:
# Base class. Use a descriptor to set a vlaue
class Descriptor:
	def __init__(self, name=None, **opts):
		self.name = name
		for key, value in opts.items():
			setattr(self, key, value)

	def __set__(self, instance, value):
		instance.__dict__[self.name] = value

# Descriptor for enforcing types
class Typed(Descriptor):
	expected_type = type(None)

	def __set__(self, instance, value):
		if not isinstance(value, self.expected_type):
			raise TypeError('expected '+ str(self.expected_type))
		super().__set__(instance, value)

# Descriptor for enforcing values
class Unsigned(Descriptor):
	def __set__(self, instance, value):
		if value < 0:
			raise ValueError('Expected >= 0')
		super().__set__(instance, value)

class MaxSized(Descriptor):
	def __init__(self, name=None, **opts):
		if 'size' not in opts:
			raise TypeError('Missing size option')
		super().__init__(name, **opts)
	def __set__(self, instance, value):
		if len(value) >= self.size:
			raise ValueError('Size must be < ' + str(self.size))
		super().__set__(instance, value)

class Integer(Typed):
	expected_type = int

class UnsignedInteger(Integer, Unsigned):
	pass

class Float(Typed):
	expected_type = float

class UnsignedFloat(Float, Unsigned):
	pass

class String(Typed):
	expected_type = str

class SizedString(String, MaxSized):
	pass




In [ ]:
class Stock:
	# Specify constraints
	name = SizedString('name',size=8)
	shares = UnsignedInteger('shares')
	price = UnsignedFloat('price')

	def __init__(self, name, shares, price):
		self.name = name
		self.shares = shares
		self.price = price
		

In [ ]:
# Class decoratir to apply constraints
from pydoc import describe


def check_attributes(**kwargs):
	def decorate(cls):
		for key, value in kwargs.items():
			if isinstance(value, Descriptor):
				value.name = key
				setattr(cls, key, value)
			else :
				setattr(cls, key, value(key))
		return cls
	return decorate

# Example
@check_attributes(name=SizedString(size=8),
shares=UnsignedInteger,
price = UnsignedFloat)
class Stock:
	def __init__(self, name, shares, price):
		self.name = name
		self.shares = shares
		self.price = price

In [3]:
# A metaclass that applies checking
class checkmeta(type):
	def __new__(cls, clsname, bases, methods):
		# Attach attribute name to the descriptors
		for key, value in methods.items():
			if isinstance(value, Descriptor):
				value.name = key
		return type.__new__(cls, clsname, bases, methods)

class Stock(metaclass=checkmeta):
	name = SizedString(size=8)
	shares = UnsignedInteger()
	price = UnsignedFloat()
	def __init__(self, name, shares, price):
		self.name = name
		self.shares = shares
		self.price = price

In [4]:
# Base class. Use a descriptor to set the value
class Descriptor:
	def __init__(self, name=None, **opts):
		self.name = name
		for key, value in opts.items():
			setattr(self, key, value)

	def __set__(self, instance, value):
		instance.__dict__[self.name] = value

# Decorator for applying type checking
def Typed(expected_type, cls=None):
	if cls is None:
		return lambda cls: Typed(expected_type, cls)
	
	super_set = cls.__set__
	def __set__(self, instance, value):
		if not isinstance(value, expected_type):
			raise TypeError('expected ' + str(expected_type))
		super_set(self, instance, value)
	cls.__set__ = __set__
	return cls

# Decorator for unsigned values
def Unsigned(cls):
	super_set = cls.__set__

	def __set__(self, instance, value):
		if value < 0:
			raise ValueError('Expected >= 0')
		super_set(self, instance, value)

	cls.__set__ = __set__
	return cls

# Decorator for allowing sized values 
def MaxSized(cls):
	super_init = cls.__init__
	def __init__(self, name=None, **opts):
		if 'size' not in opts:
			raise TypeError('missing size option')
		super_init(self, name, **opts)
	cls.__init__ = __init__

	super_set = cls.__set__

	def __set__(self, instance, value):
		if len(value) >= self.size:
			raise ValueError('size must be < ' + str(self.size))
		super_set(self, instance, value)
	cls.__set__ = __set__
	return cls

# Specialized descriptors
@Typed(int)
class Integer(Descriptor):
	pass

@Unsigned
class UnsignedInteger(Integer):
	pass

@Typed(float)
class Float(Descriptor):
	pass



In [9]:
import collections
import bisect

class SortedItems(collections.abc.Sequence):
	def __init__(self, initial=None):
		self._items = sorted(initial) if initial is not None else []

	# Required sequence methods
	def __getitem__(self, index):
		return self._items[index]

	def __len__(self):
		return len(self._items)
		
	# Method for adding an item in the right location
	def add(self, item):
		bisect.insort(self._items, item)

items = SortedItems([5,3,1,4])

list(items)

[1, 3, 4, 5]

In [ ]:
class Items(collections.abc.MutableSequence):
	def __init__(self, initial=None):
		self._items = list(initial) if intial is not None else []
	
	# Required sequence methods
	def __getitem__(self, index):
		print('Getting ', index)
		return self._items[index]

	def __setitem__(self, index, value):
		print('Setting:', index, value)
		self._items[index] = value

	def __delitem__(self, index):
		print('Deleting:', index)
		del self._items[index]
	
	def insert(self, index, value):
		print('Inserting:', index, value)
		self._items.insert(index, value)

	def __len__(self):
		print('Len')
		return len(self._items)
	

In [3]:
class A:
	def spam(self, x):
		pass

	def foo(self):
		pass

class B:
	def __init__(self):
		self._a=A()

	def spam(self, x):
		# Delegate to the inaternal self._a instance
		return self._a.spam(x)

	def foo(self):
		# Delegate to the internal self._a instance
		return self._a.foo()

	def bar(self):
		pass

# Use __getattr__() method
class B:
	def __init__(self):
		self._a = A()

	def bar(self):
		pass

	# Expose all of the methods defined on class A
	def __getattr__(self, name):
		return getattr(self._a, name)

b = B()

b.bar()

b.spam(42) # Calls B.__getattr__('spam') and delegates to A.spam



In [4]:
# A proxy class that wraps around another object, but
# exposes its public attributes

class Proxy:
	def __init__(self, obj):
		self._obj = obj

	# Delegate attribute lookup to internal obj
	def __getattr__(self, name):
		print('getattr:', name)
		return getattr(self._obj, name)

	# Delegate attribute assignment
	def __setattr__(self, name, value):
		if name.startswith('_'):
			super().__setattr__(name, value)
		else:
			print('setattr:', name, value)
			setattr(self._obj, name, value)
		
	# Delegate attribute deletion
	def __delattr__(self, name):
		if name.startswith('_'):
			super().__delattr__(name)
		else:
			print('delattr:', name)
			delattr(self._obj, name)

class Spam:
	def __init__(self, x):
		self.x = x

	def bar(self, y):
		print('Spam.bar:', self.x,y)

s = Spam(2)

p = Proxy(s)

print(p.x)

p.bar(3)

p.x = 37

getattr: x
2
getattr: bar
Spam.bar: 2 3
setattr: x 37


In [2]:
import time

class Date:
	# Primary constructor
	def __init__(self, year, month, day):
		self.year = year
		self.month = month
		self.day = day

	# Alternate constructor
	@classmethod
	def today(cls):
		t = time.localtime()
		return cls(t.tm_year, t.tm_mon, t.tm_mday)

a = Date(2012,12,21)

b = Date.today()


In [3]:
a, b

(<__main__.Date at 0x7f004db876a0>, <__main__.Date at 0x7f004db86b60>)

In [4]:
class LoggedMappingMixin:
	'''
	Add logging to get/set/delete operations for debugging
	'''

	__slots__ = ()

	def __getitem__(self, key):
		print('Getting ' + str(key))
		return super().__getitem__(key)

	def __setitem__(self, key, value):
		print('Setting {} = {!r}'.format(key, value))
		return super().__setitem__(key,value)

	def __delitem__(self, key):
		print('Deleting '+ str(key))
		return super().__delitem__(key)

class SetOnceMappingMixin:
	'''
	Only allow a key to be set once
	'''

	__slots__ = ()

	def __setitem__(self, key, value):
		if key in self:
			raise KeyError(str(key) + ' already set')
		return super().__setitem__(key, value)

class StringKeyMappingMixin:
	'''
	Restrict key to strings only
	'''

	__slots__ = ()

	def __setitem__(self, key, value):
		if not isinstance(key, str):
			raise TypeError('key must be strings')
		return super().__setitem__(key, value)
	


In [6]:
class LoggedDict(LoggedMappingMixin, dict):
	pass

d = LoggedDict()

d['x'] = 23

d['x']

del d['x']

Setting x = 23
Getting x
Deleting x


In [7]:
from collections import defaultdict
class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
	pass

d = SetOnceDefaultDict(list)
d['x'].append(2)
d['y'].append(3)

d['x'].append(10)

d['x'] = 23

KeyError: 'x already set'

In [11]:
from re import L


class RestrictKeyMixin:
	def __init__(self, *args, _restrict_key_type, **kwargs):
		self.__restrict_key_type = _restrict_key_type
		super().__init__(*args, **kwargs)

	def __setitem__(self, key, value):
		if not isinstance(key, self.__restrict_key_type):
			raise TypeError('Key must be ' + str(self.__restrict_key_type))
		super().__setitem__(key, value)

class RDict(RestrictKeyMixin, dict):
	pass

d = RDict(_restrict_key_type=str)
e = RDict([('name', 'Dave'), ('n', 37)], _restrict_key_type=str)
f = RDict(name='Dave', n=37, _restrict_key_type=str)

f['42'] = 10

f

{'name': 'Dave', 'n': 37, '42': 10}

In [ ]:
def LoggedMapping(cls):
	cls_getitem = cls.__getitem__
	cls_setitem = cls.__setitem__
	cls_delitem = cls.__delitem__

	def __getitem__(self, key):
		print('Getting ' + str(key))
		return cls_getitem(self, key)

	def __setitem__(self, key, value):
		print('Setting {} = {!r}'.format(key, value))
		return cls_setitem(self, key, value)

	def __delitem__(self, key):
		print('Deleting ', + str(key))
		return cls_delitem(self, key)

	cls.__getitem__ = __getitem__
	cls.__setitem__ = __setitem__
	cls.__delitem__ = __delitem__
	return cls

@LoggedMapping
class LoggedDict(dict):
	pass

In [12]:
class Connection:
	def __init__(self):
		self.new_state(ClosedConnectionState)

	def new_state(self, newstate):
		self._state = newstate

	def read(self):
		return self._state.read(self)

	def write(self, data):
		return self._state.write(self, data)

	def open(self):
		return self._state.open(self)

	def close(self):
		return self._state.close(self)

# Connection state base class
class ConnectionState:
	@staticmethod
	def read(conn):
		raise NotImplementedError()

	@staticmethod
	def write(conn, data):
		raise NotImplementedError()

	@staticmethod
	def open(conn):
		raise NotImplementedError()

	@staticmethod
	def close(conn):
		raise NotImplementedError()

# Implementation of different states
class ClosedConnectionState(ConnectionState):
	@staticmethod
	def read(conn):
		raise RuntimeError('Not open')

	@staticmethod
	def write(conn, data):
		raise RuntimeError('Not open')

	@staticmethod
	def open(conn):
		conn.new_state(OpenConnectionState)

	@staticmethod
	def close(conn):
		raise RuntimeError('Already closed')

class OpenConnectionState(ConnectionState):
	@staticmethod
	def read(conn):
		print('reading')

	@staticmethod
	def write(conn, data):
		print('writing')

	@staticmethod
	def open(conn):
		raise RuntimeError('Already open')

	@staticmethod
	def close(conn):
		conn.new_state(ClosedConnectionState)

		
	

In [16]:
c = Connection()

c._state


c.open()
c.read()

reading


In [2]:
import math

class Point:
	def __init__(self, x, y):
		self.x = x
		self.y = y

	def __repr__(self):
		return 'Point({!r:}, {!r:})'.format(self.x, self.y)

	def distance(self, x, y):
		return math.hypot(self.x - x, self.y -y)

p = Point(2,3)

d = getattr(p, 'distance')(0, 0)

d

3.605551275463989

In [4]:
class NodeVisitor:
	def visit(self, node):
		methname = 'visit_' + type(node).__name__
		meth = getattr(self, methname, None)
		if meth is None:
			meth = self.generic_visit
		return meth(node)

	def generic_visit(self, node):
		raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

class Evaluator(NodeVisitor):
	def visit_Number(self, node):
		return node.value

	def visit_Add(self, node):
		return self.visit(node.left) + self.visit(node.right)

	def visit_Sub(self, node):
		return self.visit(node.left) - self.visit(node.right)

	def visit_Mul(self, node):
		return self.visit(node.left) * self.visit(node.right)

	def visit_Div(self, node):
		return self.visit(node.left) / self.visit(node.right)

	def visit_Negate(self, node):
		return -node.operand

	

In [6]:
import sys
sys.setrecursionlimit(10000)


In [8]:
sys.getrecursionlimit()

10000

In [3]:
import types 

class Node:
	pass

class NodeVisitor:
	def visit(self, node):
		stack = [node]
		last_result = None
		while stack:
			try:
				last = stack[-1]
				if isinstance(last, types.GeneratorType):
					stack.append(last.send(last_result))
					last_result = None
				elif isinstance(last, Node):
					stack.append(self._visit(stack.pop()))
				else:
					last_result = stack.pop()
			except StopIteration:
				stack.pop()
		return last_result

	def _visit(self, node):
		methname='visit_'+type(node).__name__
		meth = getattr(self, methname, None)
		if meth is None:
			meth = self.generic_visit
		return meth(node)

	def generic_visie(self, node):
		raise RuntimeError('No {} method'.format('visit_'+type(node).__name__))
		

In [4]:
class UnaryOperator(Node):
	def __init__(self, operand):
		self.operand = operand

class BinaryOperator(Node):
	def __init__(self, left, right):
		self.left = left
		self.right = right

class Add(BinaryOperator):
	pass

class Sub(BinaryOperator):
	pass

class Mul(BinaryOperator):
	pass

class Div(BinaryOperator):
	pass

class Negate(UnaryOperator):
	pass

class Number(Node):
	def __init__(self,value):
		self.value = value



In [5]:
class Evaluator(NodeVisitor):
	def visit_Number(self, node):
		return node.value

	def visit_Add(self, node):
		return self.visit(node.left) + self.visit(node.right)

	def visit_Sub(self, node):
		return self.visit(node.left) - self.visit(node.right)

	def visit_Mul(self, node):
		return self.visit(node.left) * self.visit(node.right)

	def visit_Div(self, node):
		return self.visit(node.left) / self.visit(node.right)

	def visit_Negate(self, node):
		return -self.visit(node.operand)


if __name__ == '__main__':
	# 1 + 2 * (3-4) / 5
	t1 = Sub(Number(3), Number(4))
	t2 = Mul(Number(2), t1)
	t3 = Div(t2, Number(5))
	t4 = Add(Number(1), t3)

	#Evaluate it
	e = Evaluator()
	print(e.visit(t4))
	

0.6


In [6]:
class Evaluator(NodeVisitor):
	def visit_Number(self, node):
		return node.value

	def visit_Add(self, node):
		yield (yield node.left) + (yield node.right)

	def visit_Sub(slef, node):
		yield (yield node.left) - (yield node.right)

	def visit_Mul(self, node):
		yield (yield node.left) * (yield node.right)

	def visit_Div(self, node):
		yield (yield node.left) / (yield node.right)

	def visit_Negate(self, node):
		yield - (yield node.operand)

a = Number(0)

for n in range(1,1000000):
	a = Add(a, Number(n))


e = Evaluator()

print(e.visit(a))



499999500000


In [ ]:
class Evaluator(NodeVisitor):
	...
	def visit_Add(self, node):
		print('Add:', node)
		lhs = yield node.left
		print('left=', lhs)

		rhs = yield node.right
		print('right=', rhs)

		yield lhs + rhs
	...

In [8]:
import weakref

class Node:
	def __init__(self, value):
		self.value = value
		self._parent = None
		self.children = []

	def __repr__(self):
		return 'Node({!r:})'.format(self.value)

	# Property that manages the parent as a weak-reference
	@property
	def parent(self):
		return self._parent if self._parent is None else self._parent()

	@parent.setter
	def parent(self, node):
		self._parent = weakref.ref(node)

	def add_children(self, child):
		self.children.append(child)
		child.parent = self

root = Node('parent')
c1 = Node('child')
root.add_children(c1)

print(c1.parent)

del root
print(c1.parent)

Node('parent')
None
